In [1]:
# --- Import Libraries ---------------------------------------------------------
library(ggplot2)
library(ggdist)
library(patchwork)
library(dplyr)
library(Cairo)
library(glmmTMB)
library(lmerTest)
library(emmeans)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: lme4

Loading required package: Matrix


Attaching package: ‘lmerTest’


The following object is masked from ‘package:lme4’:

    lmer


The following object is masked from ‘package:stats’:

    step


Welcome to emmeans.
Caution: You lose important information if you filter this package's results.
See '? untidy'



In [ ]:
# --- Load Data ------------------------------------------------------------------
data <- read.csv("data/fullResults.csv", header = TRUE)

# Create binary anchor and alignment labels
data <- data %>%
  mutate(
    anchor = ifelse(anchorCategory == "anchor", "anchor", "non-anchor"),
    alignment = ifelse(alignmentCategory == "aligned", "aligned", "unaligned")
  )

# Create anchorType (special anchors 25 and 50)
data <- data %>%
  mutate(anchorType = case_when(
    selectedPart == 25 ~ "25",
    selectedPart == 50 ~ "50",
    TRUE ~ "non-anchor"
  ))

# Factorize main grouping variables
data$alignment <- factor(data$alignment, levels = c("aligned", "unaligned"))
data$anchor <- factor(data$anchor, levels = c("anchor", "non-anchor"))
data$chartType <- factor(data$chartType, levels = c("pie", "line"))

# Relabel factor levels for pretty plotting
levels(data$alignment) <- c("Aligned", "Unaligned")
levels(data$anchor) <- c("Anchor", "Non-Anchor")
levels(data$chartType) <- c("Pie", "Stacked Bar")

# Create numeric contrast-coded variables for modeling
data <- data %>%
  mutate(
    alignment_num = ifelse(alignment == "Aligned", -1, 1),
    anchor_num = ifelse(anchor == "Anchor", -1, 1),
    chartType_num = ifelse(chartType == "Pie", -1, 1)
  )

# Factorize detailed categories (alignmentCategory, anchorCategory)
data$alignmentCategory <- factor(data$alignmentCategory, levels = c("aligned", "near-aligned", "far-from-aligned"))
levels(data$alignmentCategory) <- c("Aligned", "Near Aligned", "Far from Aligned")

data$anchorCategory <- factor(data$anchorCategory, levels = c("anchor", "near-anchor", "far-anchor"))
levels(data$anchorCategory) <- c("Anchor", "Near Anchor", "Far from Anchor")

# Calculate rounding variables
data$distToNearest5 <- abs(data$selectedPart - round(data$selectedPart / 5) * 5)
data$distToNearest10 <- abs(data$selectedPart - round(data$selectedPart / 10) * 10)

In [ ]:
# --- Define Constants ---------------------------------------------------------
blue <- "#336199"
lightblue <- "#5688C7"
red <- "#E74236"
lightred <- "#EE766D"
green <- "#00A35F"
lightgreen <- "#0ACC7B"

lisAbsError <- c(-0.5, 10.5)
size <- 0.5
baseSize <- 28
timeLims = c(0, 15)
errorLims = c(-0.5, 10.5)

breaksAbsError <- seq(floor(min(data$absError)) - 0.5, ceiling(max(data$absError)) + 0.5, by = 1)

In [4]:
# --- Build Accuracy Model --------------------------------------------------------------
modelAccuracy <- glmmTMB(
  absError ~ chartType_num * anchor_num * alignment_num +
    (1 + anchor_num + alignment_num + chartType_num | userID),
  data = data,
  family = nbinom2(link = "log")
)
summary(modelAccuracy)

 Family: nbinom2  ( log )
Formula:          absError ~ chartType_num * anchor_num * alignment_num + (1 +  
    anchor_num + alignment_num + chartType_num | userID)
Data: data

      AIC       BIC    logLik -2*log(L)  df.resid 
  23202.9   23328.9  -11582.5   23164.9      5593 

Random effects:

Conditional model:
 Groups Name          Variance Std.Dev. Corr              
 userID (Intercept)   0.49225  0.7016                     
        anchor_num    0.13723  0.3704   -0.92             
        alignment_num 0.01488  0.1220   -0.69  0.52       
        chartType_num 0.01849  0.1360    0.18 -0.19 -0.35 
Number of obs: 5612, groups:  userID, 60

Dispersion parameter for nbinom2 family (): 1.89 

Conditional model:
                                        Estimate Std. Error z value Pr(>|z|)
(Intercept)                             0.266751   0.097208   2.744  0.00607
chartType_num                           0.041128   0.034461   1.193  0.23269
anchor_num                              0.64368

In [5]:
# --- Build Speed Model --------------------------------------------------------------
modelSpeed <- glmmTMB(
  responseTime ~ chartType_num * anchor_num * alignment_num +
    (1 + anchor_num + alignment_num + chartType_num | userID),
  data = data,
  family = Gamma(link = "log")
)
summary(modelSpeed)

 Family: Gamma  ( log )
Formula:          
responseTime ~ chartType_num * anchor_num * alignment_num + (1 +  
    anchor_num + alignment_num + chartType_num | userID)
Data: data

      AIC       BIC    logLik -2*log(L)  df.resid 
  27055.1   27181.1  -13508.5   27017.1      5593 

Random effects:

Conditional model:
 Groups Name          Variance  Std.Dev. Corr              
 userID (Intercept)   0.1248288 0.35331                    
        anchor_num    0.0019921 0.04463   0.12             
        alignment_num 0.0007469 0.02733   0.31 -0.44       
        chartType_num 0.0033118 0.05755   0.16  0.35  0.04 
Number of obs: 5612, groups:  userID, 60

Dispersion estimate for Gamma family (sigma^2): 0.147 

Conditional model:
                                         Estimate Std. Error z value Pr(>|z|)
(Intercept)                             1.8728939  0.0463525   40.41  < 2e-16
chartType_num                           0.0008645  0.0110938    0.08  0.93788
anchor_num                     

In [ ]:
# --- Predict from Model -------------------------------------------------------
predsAccuracy <- as.data.frame(emmeans(modelAccuracy, ~ alignment_num * anchor_num * chartType_num))
predsSpeed <- as.data.frame(emmeans(modelSpeed, ~ alignment_num * anchor_num * chartType_num))

In [ ]:
make_plot <- function(data, preds_df, chart_type, xvar, xlims, adjust_val = 1.5) {
  preds_df <- preds_df %>%
    mutate(
      alignment = ifelse(alignment_num == -1, "Aligned", "Unaligned"),
      anchor = ifelse(anchor_num == -1, "Anchor", "Non-Anchor"),
      chartType = ifelse(chartType_num == -1, "Pie", "Stacked Bar")
    )
  
  # Filter data and predictions
  plot_data <- data %>% filter(chartType == chart_type)
  preds_data <- preds_df %>% filter(chartType == chart_type)
  
  # Choose raw-data slab type
  slab_layer <- if (xvar == "absError") {
    stat_histinterval(
      breaks = breaksAbsError,
      slab_alpha = 0.5,
      point_alpha = 0,
      interval_alpha = 0,
      color = green,
      fill = lightgreen,
      height = 0.95
    )
  } else {
    stat_halfeye(
      slab_alpha = 0.5,
      point_size = 3,
      point_alpha = 0,
      interval_alpha = 0,
      color = green,
      fill = lightgreen,
      adjust = adjust_val,
      height = 0.95
    )
  }
  
  # Build plot
  p <- ggplot(
    plot_data,
    aes(
      y = interaction(alignment, anchor, sep = " + "),
      x = .data[[xvar]]
    )
  ) +
    slab_layer +
    geom_segment(
      data = preds_data,
      aes(
        y = interaction(alignment, anchor, sep = " + "),
        yend = interaction(alignment, anchor, sep = " + "),
        x = exp(asymp.LCL),
        xend = exp(asymp.UCL)
      ),
      color = green,
      linewidth = 2,
      inherit.aes = FALSE
    ) +
    geom_line(
      data = preds_data,
      aes(
        y = interaction(alignment, anchor, sep = " + "),
        x = exp(emmean),
        group = 1
      ),
      color = green,
      linewidth = 1.5,
      inherit.aes = FALSE
    ) +
    theme_minimal(base_size = baseSize) +
    coord_cartesian(xlim = xlims) +
    scale_y_discrete(expand = expansion(mult = c(0, 0))) +
    labs(
      y = NULL,
      x = ifelse(xvar == "absError", "Absolute Error", "Response Time (s)")
    )
  
  # Only adjust axis ticks if absError
  if (xvar == "absError") {
    p <- p + scale_x_continuous(breaks = c(0, 5, 10))  # only label 0, 5, 10
  }
  
  return(p)
}

: 

In [ ]:
accuracyPie <- make_plot(data, predsAccuracy, chart_type = "Pie", xvar = "absError", xlims = errorLims)
speedPie    <- make_plot(data, predsSpeed, chart_type = "Pie", xvar = "responseTime", xlims = timeLims)
accuracyLine <- make_plot(data, predsAccuracy, chart_type = "Stacked Bar", xvar = "absError", xlims = errorLims)
speedLine    <- make_plot(data, predsSpeed, chart_type = "Stacked Bar", xvar = "responseTime", xlims = timeLims)

width <- unit(6.66, "in")
height <- unit(2.33, "in")
options(repr.plot.width = width, repr.plot.height = height)

speedPie <- speedPie + theme(
  axis.title.y = element_blank(),
  axis.text.y = element_blank(),
  axis.ticks.y = element_blank()
)
speedLine <- speedLine + theme(
  axis.title.y = element_blank(),
  axis.text.y = element_blank(),
  axis.ticks.y = element_blank()
)

# Build your full plot
full_plot <- accuracyPie + speedPie + accuracyLine + speedLine +
  plot_layout(guides = "collect", ncol = 4)

# Open SVG device
Cairo::CairoSVG("teaser.svg", width = width, height = height)

# Draw plot
print(full_plot)

# Close device
dev.off()
full_plot

pdf 
  2